In [1]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv("./TPM/merged_tpm_csv.txt",index_col=0,header=0, sep=",")
df.head()

,Morex.Ca1,Morex.Ca2,Morex.Ca3,Morex.Co1,Morex.Co2,Morex.Co3,Morex.In1,Morex.In2,Morex.In3,Morex.Ro1,Morex.Ro2,Morex.Ro3,Morex.Sh1,Morex.Sh2,Morex.Sh3
Akashinriki_chr1HG00001,13.712022,11.519797,10.863549,18.360678,17.522969,13.801466,37.000150,28.384373,39.937753,27.414077,21.464482,23.375211,21.460764,17.167022,18.348173
Akashinriki_chr1HG00002,6.459039,8.251093,7.020342,6.055621,5.906528,5.233625,23.549376,20.293402,30.911823,5.996557,7.212982,6.504028,9.215345,10.087120,8.748190
Akashinriki_chr1HG00003,320.331153,253.217507,316.915028,320.557993,303.585931,282.454629,244.705528,139.244472,282.163271,367.158223,367.015514,363.344045,410.299341,352.515228,366.260268
Akashinriki_chr1HG00004,0.352011,0.219599,0.319982,0.010286,0.021031,0.000000,7.406792,7.344697,6.770226,0.000000,0.000000,0.000000,0.000000,0.016698,0.058996
Akashinriki_chr1HG00005,19.132994,21.453412,24.595541,11.764898,9.262669,11.117155,38.379649,25.286995,29.475611,13.466928,14.190814,15.470952,21.069363,23.672791,26.028973


In [7]:
df.dtypes

Morex.Ca1    float64
Morex.Ca2    float64
Morex.Ca3    float64
Morex.Co1    float64
Morex.Co2    float64
Morex.Co3    float64
Morex.In1    float64
Morex.In2    float64
Morex.In3    float64
Morex.Ro1    float64
Morex.Ro2    float64
Morex.Ro3    float64
Morex.Sh1    float64
Morex.Sh2    float64
Morex.Sh3    float64
dtype: object

In [8]:
# Get the number of columns in the DataFrame
num_columns = len(df.columns)

# Calculate the mean for every three columns
mean_df = pd.DataFrame()
for i in range(0, num_columns, 3):
    mean_df[f'Mean_{i//3}'] = df.iloc[:, i:i+3].mean(axis=1)

# Calculate the standard deviation for every three columns
std_df = pd.DataFrame()
for i in range(0, num_columns, 3):
    std_df[f'Std_{i//3}'] = df.iloc[:, i:i+3].std(axis=1)

In [9]:
mean_df.head(2)

,Mean_0,Mean_1,Mean_2,Mean_3,Mean_4
Akashinriki_chr1HG00001,12.031789,16.561704,35.107425,24.084590,18.991986
Akashinriki_chr1HG00002,7.243491,5.731925,24.918200,6.571189,9.350218


In [10]:
std_df.head(2)

,Std_0,Std_1,Std_2,Std_3,Std_4
Akashinriki_chr1HG00001,1.491661,2.426855,6.004744,3.037570,2.218091
Akashinriki_chr1HG00002,0.916630,0.437932,5.439943,0.610987,0.679578


## read gene id clusters

In [11]:
cd_hits='cd85.clstr'

cluster_dict={}
cluster_gene_list=[]
i=0

## read cd-hit output, for each cluster, store gene id in a list, and add to cluster dict using cluster name as key
with open(cd_hits,'r') as f1:
    for line in f1:
        if line.startswith(">"):
            i+=1 #count cluster
            if i==1: #first cluster no action
                cluster_name='C0'
                continue
            else: # other cluster, take following actions
                ##add last record
                cluster_dict[cluster_name] = cluster_gene_list
                
                ##reset cluster_name and gene_list
                cluster_name='C'+str(line.strip().split()[1])
                cluster_gene_list=[]
                
        else:
            gene_id=line.split()[2][1:-3]
            cluster_gene_list.append(gene_id)
        
        ##add last record
        cluster_dict[cluster_name]=cluster_gene_list

In [29]:
# Create empty data frame
group_mean_df = pd.DataFrame(index=cluster_dict.keys(), columns=mean_df.columns)
group_std_df = pd.DataFrame(index=cluster_dict.keys(), columns=mean_df.columns)

## barley pan-genome gene count
for key in cluster_dict.keys():
        #if len(cluster_dict[key]) ==2:
        #cluster_dict[key].sort() #sort item in list
        MorexV2_ID=[item for item in cluster_dict[key] if "MOREX" in item]
        OTHER_ID=[item for item in cluster_dict[key] if "MOREX" not in item]
        
        # Create empty DataFrames to store the results
        index_list = [item for item in cluster_dict[key] if "MOREXr2" not in item]
        
        # Get the mean values for the indices in the group
        group_mean = mean_df.loc[index_list].mean()

        # Calculate the standard error for the indices in the group
        group_std = mean_df.loc[index_list].sem()

        # Store the results in the respective DataFrames
        group_mean_df.loc[key] = group_mean
        group_std_df.loc[key] = group_std
        
        # Remove rows with all NaN values
        group_mean_df = group_mean_df.dropna(how='all')
        group_std_df = group_std_df.dropna(how='all')
        
        # Save results to file
        group_mean_df.to_csv("c85_cluster_mean_tpm_removeAllNA.csv")
        group_std_df.to_csv("c85_cluster_std_tpm_removeAllNA.csv")

In [27]:
# Find rows with all NaN values
rows_with_all_na = group_mean_df[group_mean_df.isna().all(axis=1)]
rows_with_all_na.shape

(28, 5)

In [28]:
# Find rows with all NaN values
rows_with_all_na = group_std_df[group_std_df.isna().all(axis=1)]
rows_with_all_na.shape

(42, 5)